In [132]:
# Imports
import os
from os.path import join as join_path
import numpy as np
rng_seed = 368
np.random.seed(rng_seed) # Random seed for reproducibility
import pandas as pd
import glob
import json
from tqdm.notebook import tqdm

In [13]:
!ls data

COVID.DATA.LIC.AGMT.pdf custom_license          metadata.readme
biorxiv_medrxiv         json_schema.txt         noncomm_use_subset
comm_use_subset         metadata.csv


In [293]:
# Perform pre-processing
# We take inspiration from https://www.kaggle.com/maksimeren/covid-19-literature-clustering/notebook#Data-Pre-processing
cord_data_dir = 'data'
cord_metadata_path = join_path(data_dir, 'metadata.csv')
cord_metadata_df = pd.read_csv(metadata_path, dtype={
    'sha': str,
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str,
    'doi': str
})
cord_metadata_df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


TODO: Comment on result

In [262]:
# Investigate artices with multiple metadata instances with same sha/paper_id.
shas = np.unique(cord_metadata_df['sha'].dropna().values)
cord_metadata_df_unique_shas = cord_metadata_df[cord_metadata_df['sha'].isin(shas)]
duplicated_df = cord_metadata_df_unique_shas[cord_metadata_df_unique_shas.duplicated(subset=['sha'], keep=False)].sort_values('sha')
duplicated_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
27527,7gm6ovet,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Vaccine biotechnology edited by James L. Bittl...,10.1016/0167-7799(89)90108-x,NaN,NaN,els-covid,NaN,1989-08-31,"Cavanagh, Dave",Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90108-x
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
41027,4k1k02r8,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-33 Detection of oseltamivir-resistant infl...,10.1016/s1386-6532(09)70129-5,NaN,NaN,els-covid,NaN,2009-09-30,"Carhan, A.; Albayrak, N.; Altas, A.B.; Uyar, Y.",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70129-5
41028,bcn6tpnx,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-34 A fast procedure for the detection of t...,10.1016/s1386-6532(09)70130-1,NaN,NaN,els-covid,NaN,2009-09-30,"Allice, V.; Milia, M.G.; Gregori, G.; Proia, M...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70130-1
41029,n8n1folf,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-35 Evaluation of two newly developed QIAsy...,10.1016/s1386-6532(09)70131-3,NaN,NaN,els-covid,NaN,2009-09-30,"Hanselle, T.; Kasper, Y.; Otte, M.; Ritter, S....",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70131-3
41030,jof3mnnk,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-36 Performance of the Qiagen Resplex II ve...,10.1016/s1386-6532(09)70132-5,NaN,NaN,els-covid,NaN,2009-09-30,"Halligan, E.; Bible, J.; Sehmi, J.; Dunn, H.; ...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70132-5
1239,fjmchbew,58be092086c74c58e9067121a6ba4836468e7ec3,CZI,The Author's Response: Case of the Index Patie...,10.3346/jkms.2020.35.e89,NaN,32080993,cc-by-nc,NaN,2020,"Lim, Jaegyun; Jeon, Seunghyun; Shin, Hyun Youn...",J Korean Med Sci,3005657121,#1576,True,noncomm_use_subset,https://doi.org/10.3346/jkms.2020.35.e89
44847,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patien...,10.3346/jkms.2020.35.e88,PMC7036343,32080992,cc-by-nc,NaN,2020-02-20,"Kim, Jin Yong",J Korean Med Sci,NaN,#1608,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
33296,mq8rwnqu,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Coronavirus and gastroenteritis in foals,10.1016/s0140-6736(75)80058-4,NaN,78197,els-covid,NaN,1975-10-25,"Bass, E.P.; Sharpee, R.L.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80058-4
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6


We observe that an article may have up to several metadata instances. There might be multiple `cord_uids`, `titles`, `authors` and `urls`. The rest of the columns seem to be the same, and by manually inspecting the urls we observe that some of the articles are "Letters to the editor" and simply the same article published under another name, even though the articles content are exactly the same. For this reason, we include all instances in an array to ensure we are not missing out on possibly important metadata.

In [302]:
# Parse article JSON
def parse_json_article(article_path: str) -> tuple:
    '''Parses a CORD-19 JSON article
    
    Args:
        article_path: JSON article path to parse
    
    Returns:
        TODO
    '''
    with open(article_path, 'r') as file:
        content = json.load(file)

        # Extract information
        paper_id = content['paper_id']
        abstract = []
        body_text = []
        
        # Abstract
        for item in content['abstract']:
            abstract.append(item['text'])
        
        # Body text
        for item in content['body_text']:
            body_text.append(item['text'])
            
        return paper_id, '\n'.join(abstract), '\n'.join(body_text)

def parse_all_cord_articles(data_dir: str, metadata_df: pd.DataFrame) -> pd.DataFrame:
    '''
    TODO: Docs
    '''
    all_cord_article_paths = glob.glob(f'{data_dir}/**/*.json', recursive=True)
    
    # Initialize DataFrame dictionary
    cord_df_dict = {
        'cord_uid': [],
        'paper_id': [],
        'title': [],
        'abstract': [],
        'authors': [],
        'body_text': [],
        'journal': [],
        'url': []
    }
    
    # Parse all JSON articles
    for i, article_path in enumerate(tqdm(all_cord_article_paths, unit='article')):
        
        # Parse article JSON
        paper_id, abstract, body_text = parse_json_article(article_path)
        
        metadatas = metadata_df[metadata_df['sha'] == paper_id]

        # Skip articles without metadata
        if len(metadatas) == 0:
            continue
        
        # Journal is same for all metadata instances
        journal = metadatas['journal'].values[0]
        
        # Some articles has multiple rows of metadata. We wish to include them all
        cord_uids = metadatas['cord_uid'].values
        authors = metadatas['authors'].values
        titles = metadatas['title'].values
        urls = metadatas['url'].values
        
        # Abstract from JSON might be empty, but metadata might contain it
        if abstract == '':
            abstract = []
            for md_abstract in metadatas['abstract'].values:
                if pd.isna(md_abstract):
                    continue
                abstract.append(md_abstract)
            if len(abstract) > 0:
                abstract = abstract[0] # Choose first abstract, all are same
            else:
                abstract = '' # Found no abstract, reset to empty string
        
        # Convert arrays with one element to the element itself
        if len(cord_uids) == 1:
            cord_uids = cord_uids[0]
        if len(authors) == 1:
            authors = authors[0]
        if len(titles) == 1:
            titles = titles[0]
        if len(urls) == 1:
            urls = urls[0]
        
        # Append result to dataframe
        cord_df_dict['cord_uid'].append(cord_uids)
        cord_df_dict['paper_id'].append(paper_id)
        cord_df_dict['title'].append(titles)
        cord_df_dict['abstract'].append(abstract)
        cord_df_dict['authors'].append(authors)
        cord_df_dict['body_text'].append(body_text)
        cord_df_dict['journal'].append(journal)
        cord_df_dict['url'].append(urls)
    
    cord_df = pd.DataFrame(cord_df_dict)
    
    # Handle possible duplicates
    cord_df.drop_duplicates(['abstract', 'body_text'], inplace=True)

    # Remove entries that contains NaN columns
    cord_df.dropna(inplace=True)
    
    return cord_df

In [303]:
# Parse all CORD-19 articles and save dataframe to file
cord_df_filename = join_path(data_dir, 'cord-19-articles.csv')

# Uncomment to parse and save CORD-19 articles
# cord_df = parse_all_cord_articles(cord_data_dir, cord_metadata_df)
# cord_df.to_csv(cord_df_filename, index=False)

In [304]:
# Load CORD-19 articles dataframe from file
cord_df = pd.read_csv(cord_df_filename)

In [305]:
cord_df.head()

,cord_uid,paper_id,title,abstract,authors,body_text,journal,url
0,z9c7qdpj,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,Evolutionary Medicine IV. Evolution and Emerge...,Abstract This article discusses how evolutiona...,"Scarpino, S.V.",The evolutionary history of humans is characte...,Encyclopedia of Evolutionary Biology,https://doi.org/10.1016/b978-0-12-800049-6.002...
1,lmvd99ew,6599ebbef3d868afac9daa4f80fa075675cf03bc,International aviation emissions to 2025: Can ...,"International aviation is growing rapidly, res...","Macintosh, Andrew; Wallace, Lailey","Sixty years ago, civil aviation was an infant ...",Energy Policy,https://doi.org/10.1016/j.enpol.2008.08.029
2,vernzjcy,eb5c7f3ff921ad6469b79cc8a3c122648204ece4,Mechanisms of diarrhoea,NaN,"Booth, I.W.; McNeish, A.S.",Acute infections of the gastrointestinal tract...,Baillière's Clinical Gastroenterology,https://doi.org/10.1016/0950-3528(93)90041-p
3,bwm1fl8f,b87b790c96c75faa22a085cb560f7b3d8e018b24,Chapter 3 Features of Host Cells Cellular and ...,"Abstract As obligate intracellular parasites, ...","Louten, Jennifer","There are three domains of life-Bacteria, Arch...",Essential Human Virology,https://doi.org/10.1016/b978-0-12-800947-5.000...
4,a25elc4a,68c0bb1989b6ca2b38da32a0d992027db39f80bc,Beijing's Hard and Soft Repression in Hong Kong,Hong Kong's new Police Commissioner Chris Tang...,"Hui, Victoria Tin-bor","It is also noteworthy that Tang, who was once ...",Orbis,https://doi.org/10.1016/j.orbis.2020.02.010
